In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib



In [5]:
# carregando o dataframe
df= pd.read_csv("../data/df_torques_falhas.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/df_torques_falhas.csv'

In [80]:
df.columns

Index(['KNR', '%', 'CLICKS', 'DEG', 'GRAD', 'NM', 'V', '\U00B0', 'KG', 'MIN',
       'MM', 'ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK',
       'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA',
       'S_GROUP_ID_-2', 'S_GROUP_ID_1', 'S_GROUP_ID_2', 'S_GROUP_ID_4',
       'S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137', 'S_GROUP_ID_140',
       'S_GROUP_ID_9830946', 'COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1',
       'COR_5T5T', 'COR_5TA1', 'COR_6K6K', 'COR_6KA1', 'COR_6UA1', 'COR_A1A1',
       'COR_K2A1', 'COR_K2K2', 'COR_O7O7', 'MOTOR_CWL', 'MOTOR_CWS',
       'MOTOR_DHS', 'MOTOR_DRP'],
      dtype='object')

In [81]:
# Converter as colunas para binário
cols = ['S_GROUP_ID_1', 'S_GROUP_ID_2', 'S_GROUP_ID_4', 'S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137', 'S_GROUP_ID_140', 'S_GROUP_ID_9830946']

# Criar a condição para converter as colunas para binário
df[cols] = (df[cols] > 0).astype(int)

# Exibir as colunas convertidas
df[cols]



,S_GROUP_ID_1,S_GROUP_ID_2,S_GROUP_ID_4,S_GROUP_ID_5,S_GROUP_ID_133,S_GROUP_ID_137,S_GROUP_ID_140,S_GROUP_ID_9830946
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
49186,1,1,1,1,0,0,0,0
49187,0,1,1,1,0,0,1,0
49188,1,1,0,1,0,0,1,0
49189,1,1,1,1,1,0,0,0


In [82]:
# Excluindo as cplunas com dados NaN
df = df.dropna()
df.shape

(40152, 49)

In [83]:
df = df.drop(columns = ['COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1','COR_5T5T', 'COR_5TA1', 'COR_6K6K', 'COR_6KA1', 'COR_6UA1', 'COR_A1A1', 'COR_K2A1', 'COR_K2K2', 'COR_O7O7', 'MOTOR_CWL', 'MOTOR_CWS','MOTOR_DHS', 'MOTOR_DRP', 'S_GROUP_ID_-2'])

Modelo para o tipo 1

In [84]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [85]:
# Converter as colunas para binário
cols = ['S_GROUP_ID_1', 'S_GROUP_ID_2','S_GROUP_ID_4','S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137','S_GROUP_ID_140', 'S_GROUP_ID_9830946']


In [86]:
# Iterando sobre cada grupo
for col in cols:
    print(f"Treinando o modelo para {col}")

    # Separando as features (X) e o target (y) para o grupo atual
    X = df.drop(columns=[col, 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
    y = df[col]

    # Separando as classes majoritária e minoritária com base no target y2
    X_majority = X[y == 0]
    X_minority = X[y == 1]

    # Definindo o número de clusters como o mínimo entre as duas classes
    n_clusters = min(len(X_majority), len(X_minority))

    # Aplicando o K-Means à classe majoritária
    kmeans = KMeans(n_clusters=n_clusters, random_state=1)
    kmeans.fit(X_majority)

    # Pegando os centróides dos clusters
    X_majority_centroids = kmeans.cluster_centers_

    # Combinando os centróides com a classe minoritária
    X = np.vstack((X_majority_centroids, X_minority))
    y = np.hstack((np.zeros(len(X_majority_centroids)), np.ones(len(X_minority))))

    # Separando em dados de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

     
    # Treinando o modelo
    model.fit(X_train, y_train)
    
    # Fazendo as previsões
    y_pred = model.predict(X_test)
    
    # Converter as probabilidades em classes binárias (0 ou 1) ajustando
    y_pred_classes= (y_pred > 0.5).astype(int)

    # Calcular as principais métricas
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)
    print(f"resultados da coluna{col}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # Matriz de Confusão
    print(confusion_matrix(y_test, y_pred_classes))

    # Relatório de Classificação
    print(classification_report(y_test, y_pred_classes))

    nome_arquivo = f'{col}.h5'  # Gera o nome do arquivo dinamicamente
    joblib.dump(model, nome_arquivo) # Salva o modelo com o nome gerado


Treinando o modelo para S_GROUP_ID_1
resultados da colunaS_GROUP_ID_1
Accuracy: 0.6150
Precision: 0.6399
Recall: 0.8695
F1-Score: 0.7373
[[ 601 2441]
 [ 651 4338]]
              precision    recall  f1-score   support

         0.0       0.48      0.20      0.28      3042
         1.0       0.64      0.87      0.74      4989

    accuracy                           0.61      8031
   macro avg       0.56      0.53      0.51      8031
weighted avg       0.58      0.61      0.56      8031

Treinando o modelo para S_GROUP_ID_2
resultados da colunaS_GROUP_ID_2
Accuracy: 0.9037
Precision: 0.9041
Recall: 0.9996
F1-Score: 0.9494
[[   1  770]
 [   3 7257]]
              precision    recall  f1-score   support

         0.0       0.25      0.00      0.00       771
         1.0       0.90      1.00      0.95      7260

    accuracy                           0.90      8031
   macro avg       0.58      0.50      0.48      8031
weighted avg       0.84      0.90      0.86      8031

Treinando o modelo